In [13]:
#Coding the Likelihood Function
import numpy as np
from scipy.stats import norm

def ar7_conditional_likelihood(params, data):
    """
    Calculate the conditional log likelihood of an AR(7) model.

    Args:
        params (list): Parameters [phi1, phi2, ..., phi7, sigma^2].
        data (array): Array of observed data.

    Returns:
        float: Conditional log likelihood value.
    """
    phi = params[:-1]
    sigma_squared = params[-1]
    n = len(data)
    conditional_ll = 0
   # Calculate conditional log likelihood for each observation starting from the 8th
    for t in range(7, n):
        # Calculate conditional mean based on past 7 observations
        mean = np.dot(phi, data[t-7:t][::-1])
        # Calculate log likelihood contribution of current observation
        conditional_ll += norm.logpdf(data[t], loc=mean, scale=np.sqrt(sigma_squared))

    return conditional_ll

def ar7_unconditional_likelihood(params, data):
    """
    Calculate the unconditional log likelihood of an AR(7) model.

    Args:
        params (list): Parameters [phi1, phi2, ..., phi7, sigma^2].
        data (array): Array of observed data.

    Returns:
        float: Unconditional log likelihood value.
    """
    phi = params[:-1]
    sigma_squared = params[-1]
    n = len(data)
    unconditional_ll = 0
# Calculate the unconditional mean of the AR(7) process
    mu = np.dot(phi, data[:7][::-1])

    # Calculate unconditional log likelihood for each observation
    for t in range(n):
        # Calculate log likelihood contribution of current observation using unconditional mean
        unconditional_ll += norm.logpdf(data[t], loc=mu, scale=np.sqrt(sigma_squared))

    return unconditional_ll

# Example:
# Define some sample data and initial parameters
data = np.random.normal(size=100)  # Sample data
initial_params = [0.5, -0.2, 0.3, -0.1, 0.2, -0.4, 0.1, 1.0]  # Initial parameters

# Calculate conditional log likelihood
conditional_ll = ar7_conditional_likelihood(initial_params, data)
print("Conditional Log Likelihood:", conditional_ll)

# Calculate unconditional log likelihood
unconditional_ll = ar7_unconditional_likelihood(initial_params, data)
print("Unconditional Log Likelihood:", unconditional_ll)

Conditional Log Likelihood: -152.25311015654904
Unconditional Log Likelihood: -252.57718175899367


In [15]:
#Maximizing the likelihood
from scipy.optimize import minimize

# Define negative log likelihood functions (to be minimized)
def neg_log_likelihood_conditional(params, data):
    return -ar7_conditional_likelihood(params, data)

def neg_log_likelihood_unconditional(params, data):
    return -ar7_unconditional_likelihood(params, data)

# Example usage:
# Define some sample data
data = np.random.normal(size=100)  # Sample data

# Initial parameter guess
initial_params = [0.5, -0.2, 0.3, -0.1, 0.2, -0.4, 0.1, 1.0]

# Maximized conditional likelihood
result_conditional = minimize(neg_log_likelihood_conditional, initial_params, args=(data,), method='Nelder-Mead')
print("Maximized Conditional Likelihood Parameters:", result_conditional.x)

# Maximized unconditional likelihood
result_unconditional = minimize(neg_log_likelihood_unconditional, initial_params, args=(data,), method='Nelder-Mead')
print("Maximized Unconditional Likelihood Parameters:", result_unconditional.x)

Maximized Conditional Likelihood Parameters: [-0.11322102 -0.09669263  0.12516164 -0.06434923 -0.06394338  0.07508277
  0.14655368  0.86626522]
Maximized Unconditional Likelihood Parameters: [ 0.07605041 -0.37926718  0.17937945 -0.11057593  0.15209992 -0.5152859
  0.12878705  0.96624656]


In [1]:
#Parameter Estimation
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Step 1: Data Preprocessing
# Load FRED-MD dataset
data = pd.read_csv('/content/current.csv')

# Assuming 'INDPRO' is the column name for the INDPRO variable
indpro = data['INDPRO']

# Calculate monthly log differences
log_diff_indpro = np.log(indpro).diff().dropna()

# Step 2: Likelihood Functions
def ar7_conditional_likelihood(params, data):
    phi, sigma_sq = params[:7], params[7]
    n = len(data)
    ll = 0
    for t in range(7, n):
        yt = data[t]
        mu_t = np.dot(phi, data[t-7:t][::-1])  # Reverse the order to align with AR(7) formula
        ll += -0.5 * (np.log(2 * np.pi * sigma_sq) + (yt - mu_t)**2 / sigma_sq)
    return ll

def ar7_unconditional_likelihood(params, data):
    phi, sigma_sq = params[:7], params[7]
    n = len(data)
    mu0 = np.mean(data[:7])  # Initial mean
    ll = -0.5 * (7 * np.log(2 * np.pi * sigma_sq) + np.sum((data[:7] - mu0)**2) / sigma_sq)
    for t in range(7, n):
        yt = data[t]
        mu_t = np.dot(phi, data[t-7:t][::-1])  # Reverse the order to align with AR(7) formula
        ll += -0.5 * (np.log(2 * np.pi * sigma_sq) + (yt - mu_t)**2 / sigma_sq)
    return ll

# Step 3: Parameter Estimation
# Initial parameter guess
initial_params = np.array([0.5, -0.2, 0.3, -0.1, 0.2, -0.4, 0.1, 1.0])

# Maximize conditional likelihood
result_conditional = minimize(ar7_conditional_likelihood, initial_params, args=(log_diff_indpro,), method='Nelder-Mead')
print("Maximized Conditional Likelihood Parameters:", result_conditional.x)

# Maximize unconditional likelihood
result_unconditional = minimize(ar7_unconditional_likelihood, initial_params, args=(log_diff_indpro,), method='Nelder-Mead')
print("Maximized Unconditional Likelihood Parameters:", result_unconditional.x)


Maximized Conditional Likelihood Parameters: [-1.05849043e+50  1.06746350e+49 -1.73462824e+49  2.64622188e+49
  2.68764340e+49 -4.69880006e+49 -3.21491762e+49  6.97645179e+50]
Maximized Unconditional Likelihood Parameters: [-1.05849043e+50  1.06746350e+49 -1.73462824e+49  2.64622188e+49
  2.68764340e+49 -4.69880006e+49 -3.21491762e+49  6.97645179e+50]


In [2]:
import numpy as np

# Function to forecast future values
def forecast_ar7(params, data, steps):
    # Extract parameters
    phi = params[:7]
    sigma = params[7]

    # Initialize forecast array with observed data
    forecast = np.copy(data)

    # Forecast future values
    for i in range(steps):
        forecast_next = np.dot(phi, forecast[-7:]) + np.random.normal(0, sigma)
        forecast = np.append(forecast, forecast_next)

    return forecast
    # Forecast using conditional likelihood parameters
conditional_forecast = forecast_ar7(result_conditional.x, log_diff_indpro, 8)
print("Conditional Forecast:", conditional_forecast[-8:])

# Forecast using unconditional likelihood parameters
unconditional_forecast = forecast_ar7(result_unconditional.x, log_diff_indpro, 8)
print("Unconditional Forecast:", unconditional_forecast[-8:])

Conditional Forecast: [ 4.31338229e+050 -1.38671687e+100  4.45818050e+149 -1.43326830e+199
  4.60783952e+248 -1.48138244e+298              inf              nan]
Unconditional Forecast: [-1.65769743e+050  5.32936067e+099 -1.71334555e+149  5.50826479e+198
 -1.77086175e+248  5.69317465e+297             -inf              nan]
